# Focault test data reduction

This document will describe step by step the process of extracting the surface profile of a telescope mirror from Foucault test data.

## Derivation of the mirror curve from test data

$Z(x) = \langle x, z(x) \rangle$

$F(x) = \langle 0, f(x) \rangle$

$T(x) = \frac{Z'(x)}{|Z'(x)|} = \langle\frac{1}{|Z'(x)|}, \frac{z'(x)}{|Z'(x)|}\rangle$

$F(x) - Z(x) = \langle 0 - x, f(x)-z(x) \rangle = \langle -x, f(x) - z(x) \rangle$

$T(x) \cdot (F(x) - Z(x)) = 0$

$\left( - \frac{x}{|Z'(x)|} + (f(x) - z(x))\frac{z'(x)}{|Z'(x)|} \right) = 0$

$(f(x) - z(x))z'(x) = x$

$z'(x) = \frac{x}{f(x) - z(x)}$

### kjhkhkhkjh